In [ ]:
# Air drawing App

import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)


canvas = None
prev_x,prev_y = 0,0

def finger_up(lms):
    fingers = []
    fingers.append(1 if lms[4][0] > lms[3][0] else 0)
    for tip in [8,12,16,20]:
        fingers.append(1 if lms[tip][1] < lms[tip-2][1] else 0)
    return fingers


        

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    h,w,_ = f.shape

    if canvas is None:
        canvas = np.zeros_like(f)


    lms = []
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            for id,lm in enumerate(hand_landmarks.landmark):
                cx,cy = int(lm.x*w),int(lm.y*h)
                lms.append((cx,cy))
            mp_draw.draw_landmarks(f,hand_landmarks,mp_hands.HAND_CONNECTIONS)
    
    if(len(lms)>=21):
        fingers = finger_up(lms)

        if fingers[1]==1 and all(f==0 for i,f in enumerate(fingers) if i!=1):
            x,y = lms[8]
            if prev_x==0 and prev_y==0:
                prev_x,prev_y=x,y
            cv2.line(canvas,(prev_x,prev_y),(x,y),(255,0,0),5)
            prev_x,prev_y=x,y
        else:
            prev_x,prev_y=0,0

        if fingers==[1,1,1,1,1]:
            cv2.putText(f,"Clearing Canvas ....",(10,70),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.5,(0,0,255),2)
            canvas = np.zeros_like(f)
            

    gray_canvas = cv2.cvtColor(canvas,cv2.COLOR_BGR2GRAY)
    _,inv = cv2.threshold(gray_canvas,50,255,cv2.THRESH_BINARY_INV)
    inv = cv2.cvtColor(inv,cv2.COLOR_GRAY2BGR)
    f = cv2.bitwise_and(f,inv)
    f = cv2.bitwise_or(f,canvas)

    cv2.putText(f,"Air Drawing - Index finger to draw.",(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.8,(0,255,0),2)
    cv2.imshow("Air Drawing app",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()